In [19]:
import os

In [20]:
os.chdir("D:/ac/ML/NLP/Text-summarizer")
#os.chdir("../")

In [21]:
%pwd

'D:\\ac\\ML\\NLP\\Text-summarizer'

In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [23]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
           
        )

        return model_evaluation_config

In [33]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_from_disk
import pandas as pd
from nltk.translate.meteor_score import meteor_score

In [34]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Split the dataset into smaller batches that we can process simultaneously.
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, model, tokenizer, 
                                    batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                                    column_text="article", 
                                    column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        scores = []

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024, truncation=True, 
                               padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_mask=inputs["attention_mask"].to(device), 
                                       length_penalty=0.8, num_beams=8, max_length=128)
            # Length penalty ensures that the model does not generate sequences that are too long.
            
            # Decode the generated texts
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                                  clean_up_tokenization_spaces=True) 
                                 for s in summaries]
            
            for ref, pred in zip(target_batch, decoded_summaries):
                # METEOR score expects reference and hypothesis as lists of tokens
                reference = ref.split()
                prediction = pred.split()
                score = meteor_score([reference], prediction)
                scores.append(score)
            
        average_score = sum(scores) / len(scores)
        return average_score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        # Loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], model_pegasus, tokenizer, 
            batch_size=2, column_text='dialogue', column_summary='summary'
        )

        df = pd.DataFrame({'meteor': [score]}, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)



In [35]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e


[2024-07-27 12:13:29,490: INFO: common yaml file: config\config.yaml loaded successfully]
[2024-07-27 12:13:29,492: INFO: common yaml file: Params.yaml loaded successfully]
[2024-07-27 12:13:29,493: INFO: common created directory at: artifacts]
[2024-07-27 12:13:29,493: INFO: common created directory at: artifacts/model_evaluation]


100%|██████████| 5/5 [04:02<00:00, 48.51s/it]
